# **DASA CS-2: [TAS: Tele Assistance System]**

In [1]:
# -*- coding: utf-8 -*-
# Native imports
import os
import re
import sys
import time

# Third-party imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# PyDASA imports
# config module
from src.pydasa.utils import config
# FDU modules
from src.pydasa.core.fundamental import Dimension
# FDU regex management
from src.pydasa.dimensional.framework import DimScheme
# Variable and Variable modules
from src.pydasa.core.parameter import Variable
# Dimensional Matrix Modelling module
from src.pydasa.dimensional.model import DimMatrix
# sensitivity analysis modules
from src.pydasa.analysis.scenario import DimSensitivity
from src.pydasa.handlers.influence import SensitivityHandler
# Monte Carlo Simulation modules
from src.pydasa.analysis.simulation import MonteCarloSim
from src.pydasa.handlers.practical import MonteCarloHandler

# FIXME old imports, delete later!!!
# from Src.PyDASA.Measure.fdu import FDU
# from Src.PyDASA.Measure.params import Parameter, Variable
# from Src.PyDASA.Utils.cstm import RegexManager
# import Src.PyDASA.Utils.cfg as PyDASAcfg
# from Src.PyDASA.Model.dim import DimensionalModel, DimensionalAnalyzer
# from Src.PyDASA.Simul.sens import Sensitivity
# from Src.PyDASA.Simul.rprt import SensitivityAnalysis
# # from Src.PyDASA.Simul.moca import MonteCarloSimulation

## **Project Overview**

- TAS offers telehealth services to patients at home.
- It uses third-party services for medical analysis, emergency alarms, and drug delivery.
- Self-adaptation deals with uncertainty (changing service quality, patient needs).
- The system uses runtime models, simulations, and a MAPE-K feedback loop (ActivFORMS runtime environment) to adapt proactively​

## **Software Architecture Summary**

### **Base Design**
Service-oriented architecture (SOA) + MAPE-K feedback loop
- **Service-oriented architecture (SOA):** Each Comp is a service that can be replaced or reconfigured.
- **MAPE-K feedback loop:** Monitors the system, analyzes its state, plans adaptations, and executes them based on knowledge stored in the Knowledge Base.

Dividing TAS into Target and Controller Parts (MAPE-K Structure)

| Subsystem | Comps |
|:---|:---|
| **Target System (Managed System)** | - **TAS Core Services:** Vital Parameters Monitor, Medication Management, Emergency Response Trigger <br> - **Service Integrations:** Medical Analysis Services (MAS), Alarm Services (AS), Drug Services (DS) from third-party providers <br> - **Probes:** Instruments to collect data about vital signs, service invocations, failures, and latencies <br> - **Effectors:** Mechanisms to replace or reconfigure services dynamically |
| **Controller (Managing System)** | - **Monitor:** Observes system state, updates runtime models (service failures, delays, costs) <br> - **Analyzer:** Runs simulations (runtime statistical model checking) on updated models to predict system qualities <br> - **Planner:** Selects best service configuration based on goals (availability, performance, cost) <br> - **Executor:** Applies adaptations (switches services, reconfigures workflows) <br> - **Knowledge Base:** Maintains models of the system, environment models, adaptation goals, quality models |

Reference: Based on MAPE-K division and ActivFORMS architecture.

### **Notes**
- TAS (Tele Assistance System) operates in a dynamic environment where service quality, availability, and user needs frequently change.
- It implements **self-adaptive mechanisms** using the MAPE-K feedback loop.
- Adaptation is critical to maintaining **reliability**, **performance**, and **compliance** with patient care standards.
- ActivFORMS provides the runtime framework for model-based adaptation using runtime models, simulations, and verified decision-making.
---

### **Functional Requirements (Reported Capabilities)**

| ID | Description |
|:-----|:---|
| RC-001 | Provide remote health support for patients with chronic conditions within their homes. |
| RC-002 | Periodically measure patients' vital signs using wearable devices and sensors. |
| RC-003 | Analyze patient data using third-party medical analysis services to assess health status. |
| RC-004 | Invoke pharmacy services to deliver medication if analysis indicates a treatment change is needed. |
| RC-005 | Adjust medication dosage dynamically based on updated health analysis. |
| RC-006 | Send automatic alarms and alerts to emergency services during critical situations. |
| RC-007 | Dynamically integrate new third-party services into existing workflows to enhance system functionality. |
| RC-008 | Adapt to changing healthcare goals by modifying workflows and service selections in real-time. |
| RC-009 | Ensure system reliability by switching to equivalent services when current ones fail or degrade. |
| RC-010 | Continuously monitor service performance and adjust workflows to optimize reliability, performance, and cost. |

---

### **Quality Scenarios (QS)**

| ID | Quality Attribute | Sub-Category | Scenario Description |
|:-----|:---|:---|:---|
| QS-001 | Availability | Fault Tolerance | TAS must detect third-party service failures and switch to a backup service within 30 seconds. |
| QS-002 | Performance | Response Time | TAS must process health analysis results and issue alerts within 2.5 seconds. |
| QS-003 | Interoperability | Syntactic/Semantic | TAS must successfully integrate new third-party services dynamically without downtime. |
| QS-004 | Usability | Learnability | Patients must access and understand health data via the TAS interface within 5 minutes of onboarding. |
| QS-005 | Security | Confidentiality | Patient data must be encrypted during transmission and storage, ensuring HIPAA compliance. |
| QS-006 | Adaptability | Dynamic Reconfiguration | TAS must reconfigure workflows dynamically based on real-time patient data and healthcare goals. |

---

### **Adaptation Scenarios (AS)**

| ID | Scenario | Description | Adaptation Strategy |
|:-------|:---|:---|:---|
| AS-001 | Service Failure | A third-party service fails to respond, impacting TAS operations. | Switch to an equivalent service; use redundancy (invoke multiple services simultaneously if needed). |
| AS-002 | Variation in Service Response Time | A third-party service exceeds expected response times, degrading TAS performance. | Switch to a more performant service; prioritize services based on real-time response metrics. |
| AS-003 | New Service Introduction | A new third-party service becomes available. | Dynamically recognize and integrate new services into the registry and workflows. |
| AS-004 | Changing Goals | Healthcare objectives or patient needs change over time. | Adjust workflows and select appropriate services to meet updated goals. |
| AS-005 | Wrong Operation Sequence | Errors or inefficiencies in workflow execution. | Redesign and reconfigure the workflow sequence to ensure compliance with functional requirements. |

---

#### **FDU (Fundamental Design Unit)**

In [2]:
# setting up custom FDU for software architecture
tas_fwk = "SOFTWARE"
tas_scm = DimScheme(_sym="FDU_{TAS}",
                        _alias="fdu_tas",
                        _idx=0,
                        name="TAS FDUs",
                        _fwk=tas_fwk,
                        description="FDU schema for the TAS case study.")
# print(tas_scm)
print("=== TAS Fundamental Dimenional Unit (FDU) Schema ===")
for dim in tas_scm.fdus:
    print(f"\t'{dim.sym}': {dim.name}, [{dim.unit}]")

=== TAS Fundamental Dimenional Unit (FDU) Schema ===
	'T': Time, [s]
	'D': Data, [bit]
	'E': Effort, [req]
	'C': Connectivity, [node]
	'A': Capacity, [process]


In [3]:
tas_scm.update_global_config()
print("\n==== Checking Schema Regex ====")
print("\tWKNG_DFLT_FDU_PREC_LT:", config.WKNG_FDU_PREC_LT)
print("\tWKNG_FDU_RE:", config.WKNG_FDU_RE)
print("\tWKNG_POW_RE:", config.WKNG_POW_RE)
print("\tWKNG_NO_POW_RE:", config.WKNG_NO_POW_RE)
print("\tWKNG_FDU_SYM_RE:", config.WKNG_FDU_SYM_RE)


==== Checking Schema Regex ====
	WKNG_DFLT_FDU_PREC_LT: ['T', 'D', 'E', 'C', 'A']
	WKNG_FDU_RE: ^[TDECA](\^-?\d+)?(\*[TDECA](?:\^-?\d+)?)*$
	WKNG_POW_RE: \-?\d+
	WKNG_NO_POW_RE: [TDECA](?!\^)
	WKNG_FDU_SYM_RE: [TDECA]


#### **Dimensional Variables**

Based on the QN (Queue Network) model for the component + sequenece diagrams

<!-- ![Diagram description](./images/diagram.svg) -->
<img src="./assets/img/CS-01-Health TAS/04A - Queue Network.svg"  width="1200" alt="TAS queue network component diagram" style="display: block; margin: auto;">
<!-- <img src="./images/tas_architecture.svg" width="800" alt="TAS queue network component diagram" style="display: block; margin: auto;"> -->


In [4]:
# list of variables
# all components use the queue model M/M/1/K
# arrival rate => lambda, IN, independent
# departure rate => chi/x, IN, dependent
# service rate => miu, IN, independent
# request in queue => n, CTRL ~ < 60%*K
# requests in service/servers => c, IN
# mean service time => S, OUT, dependant
# mean queue time => Q, OUT, dependant
# mean response time => W = Q + S, OUT, independent
# mean queue size => L = λ * W, OUT, dependant
# max queue size => K, independent

print("\n==== Variables ====")
# dict with variables
tas_vars ={
    # C1 -> TAS (Tele Assistance Service) 1
    # C1 = C2 = C3 = C10 = C12 = C13
    # C1 = TAS_1
    # λ1=344.83 [req/s], Χ1=344.83 [req/s], μ1=1000.00 [req/s]
    "\\lambda_{1}": Variable(_sym="\\lambda_{1}",
                                   _alias="lambda_tas_1",
                                   _fwk="SOFTWARE",
                                   name="TAS component 1 arrival 1",
                                   description="TAS arrival rate 1",
                                   relevant=True,
                                   _idx=0,
                                   _cat="IN",
                                   _units="req/s",
                                   _dims="E*T^-1",
                                   _min=0.0,
                                   _max=344.83,
                                   _mean=344.83,
                                   _std_units="req/s",
                                   _std_min=0.0,
                                   _std_max=344.83,
                                   _std_mean=344.83,
                                   _dist_type="exponential",
                                   _dist_params={"scale": 1/344.83},
                                   _dist_func=lambda: np.random.exponential(1/344.83)),
    "\\miu_{1}": Variable(_sym="\\miu_{1}",
                                   _alias="miu_tas_1",
                                   _fwk="SOFTWARE",
                                   name="TAS component 1 service rate",
                                   description="TAS service rate 1",
                                   relevant=True,
                                   _idx=1,
                                _cat="CTRL",
                                   _units="req/s",
                                   _dims="E*T^-1",
                                   _min=0.0,
                                   _max=1000.0,
                                   _mean=1000.0,
                                   _std_units="req/s",
                                   _std_min=0.0,
                                   _std_max=1000.0,
                                   _std_mean=1000.0,
                                   _dist_type="exponential",
                                   _dist_params={"scale": 1/1000},
                                   _dist_func=lambda: np.random.exponential(1/1000)),
    "K_{1}": Variable(_sym="K_{1}",
                             _alias="k_1",
                             _fwk="SOFTWARE",
                             name="TAS component 1 max queue size",
                             description="TAS max queue size 1",
                             relevant=True,
                             _idx=3,
                             _cat="IN",
                             _units="req",
                             _dims="E",
                             _min=99.9,
                             _max=100.1,
                             _mean=100.0,
                             _std_units="req",
                             _std_min=99.9,
                             _std_max=100.1,
                             _std_mean=100.0,
                             _dist_type="uniform",
                             _dist_params={"constant": 100},
                             _dist_func=lambda: 100),
    "n_{1}": Variable(_sym="n_{1}",
                             _alias="n_1",
                             _fwk="SOFTWARE",
                             name="TAS component 1 requests in queue",
                             description="TAS requests in queue 1",
                             relevant=True,
                             _idx=2,
                             _cat="IN",
                             _units="req",
                             _dims="E",
                             _min=0.0,
                             _max=60.0,
                             _mean=30.0,
                             _std_units="req",
                             _std_min=0.0,
                             _std_max=60.0,
                             _std_mean=30.0,
                             _dist_type="uniform",
                             _dist_params={"low": 0, "high": 60},
                             _dist_func=lambda: np.random.uniform(0, 60)),
    "W_{1}": Variable(_sym="W_{1}",
                             _alias="w_1",
                             _fwk="SOFTWARE",
                             name="TAS component 1 response time",
                             description="TAS response time 1",
                             relevant=True,
                             _idx=4,
                             _cat="OUT",
                             _units="s/req",
                             _dims="T*E^-1",
                             _min=0.0,
                             _max=60.0,
                             _mean=30.0,
                             _std_units="s/req",
                             _std_min=0.0,
                             _std_max=60.0,
                             _std_mean=30.0,
                             _dist_type="uniform",
                             _dist_params={"low": 0, "high": 60},
                             _dist_func=lambda: np.random.uniform(0, 60)),
    "L_{1}": Variable(_sym="L_{1}",
                             _alias="l_1",
                             _fwk="SOFTWARE",
                             name="TAS component 1 latency",
                             description="TAS latency 1",
                             relevant=True,
                             _idx=5,
                            _cat="CTRL",
                             _units="req",
                             _dims="E",
                             _min=0.0,
                             _max=60.0,
                             _mean=30.0,
                             _std_units="req",
                             _std_min=0.0,
                             _std_max=60.0,
                             _std_mean=30.0,
                             _dist_type="uniform",
                             _dist_params={"low": 0, "high": 60},
                             _dist_func=lambda: np.random.uniform(0, 60)),
    # # C2 = TAS_2
    # # λ2=258.62 [req/s], Χ2=258.62 [req/s], μ2=1000.00 [req/s]
    # "C_{TAS_{2}}": Variable(),
    # # C3 = TAS_3
    # # λ3=174.14 [req/s], Χ3=174.14 [req/s], μ3=1000.00 [req/s]
    # "C_{TAS_{3}}": Variable(),
    # # C4, C5, C6 -> MAS (Medical Assistance Service)
    # # C4 = MAS_1
    # # λ4=88.17 [req/s], Χ4=77.59 [req/s], μ4=180.00 [req/s]
    # "C_{MAS_{1}}": Variable(),
    # # C5 = MAS_2
    # # λ5=132.09 [req/s], Χ5=122.84 [req/s], μ5=530.00 [req/s]
    # "C_{MAS_{2}}": Variable(),
    # # C6 = MAS_3
    # # λ6=70.96 [req/s], Χ6=58.19 [req/s], μ6=150.00 [req/s]
    # "C_{MAS_{3}}": Variable(),
    # # C7, C8, C9 -> AS (Alarm Service)
    # # C7 = AS_1
    # # λ7=65.22 [req/s], Χ7=58.05 [req/s], μ7=700.00 [req/s]
    # "C_{AS_{1}}": Variable(),
    # # C8 = AS_2
    # # λ8=60.46 [req/s], Χ8=58.05 [req/s], μ8=410.00 [req/s]
    # "C_{AS_{2}}": Variable(),
    # # C9 = AS_3
    # # λ9=70.79 [req/s], Χ9=58.05 [req/s], μ9=1580.00 [req/s]
    # "C_{AS_{3}}": Variable(),
    # # C10 = TAS_4
    # # λ10=60.46 [req/s], Χ10=58.05 [req/s], μ10=410.00 [req/s]
    # "C_{TAS_{4}}": Variable(),
    # # C11 -> DS (Drug Service)
    # # C11 = DS_1
    # # λ11=179.67 [req/s], Χ11=183.62 [req/s], μ11=550.00 [req/s]
    # "C_{DS_{1}}": Variable(),
    # # C12 = TAS_5
    # # λ12=174.14 [req/s], Χ12=174.14 [req/s], μ12=1000.00 [req/s]
    # "C_{TAS_{5}}": Variable(),
    # # C13 = TAS_6
    # # λ13=170.69 [req/s], Χ13=174.44 [req/s], μ13=1000.00 [req/s]
    # "C_{TAS_{6}}": Variable(),
}





"""
C Vs. Metric	λi [req/s]	Χi [req/s]	μi [req/s]
*C1	                344.83	344.83	1000.00
*C2	                258.62	258.62	1000.00
*C3	                174.14	174.14	1000.00
C4	                88.17	77.59	180.00
C5	                132.09	122.84	530.00
C6	                70.96	58.19	150.00
C7	                65.22	58.05	700.00
C8	                60.46	58.05	410.00
C9	                70.79	58.05	1580.00
*C10	            258.62	258.62	1000.00
C11	                179.67	183.62	550.00
*C12	            174.14	174.14	1000.00
*C13	            170.69	174.44	1000.00
Sum	                157.57	153.94	776.92

Avg for all C1-C13	λi [req/s]	Χi [req/s]	μi [req/s]
f(M/M/1/∞/FIFO)	f(M/M/1/∞/FIFO)
Li [req]	Wi [s*req^(-1)]
2.333E+00	2.020E-02


"""


for i, var in enumerate(tas_vars.values()):
    var.idx = i
    print(f"Variable {var.idx}: '{var.sym}', ({var.cat})")
    print(f"\tStd Units: '{var.std_units}'")
    print(f"\tDimensionality: '{var.dims}'")



==== Variables ====
Variable 0: '\lambda_{1}', (IN)
	Std Units: 'req/s'
	Dimensionality: 'E*T^-1'
Variable 1: '\miu_{1}', (CTRL)
	Std Units: 'req/s'
	Dimensionality: 'E*T^-1'
Variable 2: 'K_{1}', (IN)
	Std Units: 'req'
	Dimensionality: 'E'
Variable 3: 'n_{1}', (IN)
	Std Units: 'req'
	Dimensionality: 'E'
Variable 4: 'W_{1}', (OUT)
	Std Units: 's/req'
	Dimensionality: 'T*E^-1'
Variable 5: 'L_{1}', (CTRL)
	Std Units: 'req'
	Dimensionality: 'E'


#### **Dimensionl Model**

In [5]:
tas_model = DimMatrix(_fwk="SOFTWARE",
                    _idx=0,
                    _framework=tas_scm)
tas_model.variables = tas_vars
print("Setting parameters for the dimensional analysis")
print(len(tas_model.variables), tas_model.variables, "\n")

tas_model.relevant_lt = tas_vars
print("Setting the relevance list for dimensional analysis")
print(len(tas_model.relevant_lt), tas_model.relevant_lt, "\n")

print(tas_model, "\n")
print(tas_model.working_fdus, "\n")

Setting parameters for the dimensional analysis
6 {'\\lambda_{1}': Variable(_sym='\\lambda_{1}', _alias='lambda_tas_1', _fwk='SOFTWARE', _idx=0, name='TAS component 1 arrival 1', description='Tas arrival rate 1', _cat='IN', _dims='E*T^-1', _units='req/s', _std_dims='T^(-1)*E^(1)', _sym_exp='T**(-1)* E**(1)', _dim_col=[-1, 0, 1, 0, 0], _min=0.0, _max=344.83, _mean=344.83, _dev=None, _std_units='req/s', _std_min=0.0, _std_max=344.83, _std_mean=344.83, _std_dev=None, _step=0.001, _std_range=array([], dtype=float64), _dist_type='exponential', _dist_params={'scale': 0.0028999797001420993}, _dist_func=<function <lambda> at 0x000001F9DB39E830>, relevant=True), '\\miu_{1}': Variable(_sym='\\miu_{1}', _alias='miu_tas_1', _fwk='SOFTWARE', _idx=4, name='TAS component 1 service rate', description='Tas service rate 1', _cat='CTRL', _dims='E*T^-1', _units='req/s', _std_dims='T^(-1)*E^(1)', _sym_exp='T**(-1)* E**(1)', _dim_col=[-1, 0, 1, 0, 0], _min=0.0, _max=1000.0, _mean=1000.0, _dev=None, _std_uni

In [6]:
# solving dimensional problem
tas_model.create_matrix()
tas_model.solve_matrix()

print(tas_model._pivot_cols, "\n")
print(tas_model, "\n")

[0, 1] 

DimMatrix(sym='', alias='', fwk='SOFTWARE', idx=0, name='Dimensional Matrix', description='', framework=DimScheme(_sym='FDU_{TAS}', _alias='fdu_tas', _fwk='SOFTWARE', _idx=0, name='TAS FDUs', description='Fdu schema for the tas case study.', _fdus=[Dimension(_sym='T', _alias='', _fwk='SOFTWARE', _idx=0, name='Time', description='Duration of an event or interval.', _unit='s'), Dimension(_sym='D', _alias='', _fwk='SOFTWARE', _idx=1, name='Data', description='Information processed by a system.', _unit='bit'), Dimension(_sym='E', _alias='', _fwk='SOFTWARE', _idx=2, name='Effort', description='Measure of computational effort/complexity.', _unit='req'), Dimension(_sym='C', _alias='', _fwk='SOFTWARE', _idx=3, name='Connectivity', description='Measure of interconnections between components.', _unit='node'), Dimension(_sym='A', _alias='', _fwk='SOFTWARE', _idx=4, name='Capacity', description='Maximum amount of data that can be stored/processed.', _unit='process')], _fdu_map={'T': Dimen

#### **Dimensionless Coefficients**

In [7]:
for k, v in tas_model.coefficients.items():
    pi = v
    print(f"Coefficient: {k} = {pi._pi_expr}")

Coefficient: \Pi_{0} = \frac{n_{1}}{K_{1}}
Coefficient: \Pi_{1} = \lambda_{1}*W_{1}
Coefficient: \Pi_{2} = \frac{\miu_{1}}{\lambda_{1}}
Coefficient: \Pi_{3} = \frac{L_{1}}{K_{1}}


#### **Sensitivity Analysis**

In [8]:
tas_sena = SensitivityHandler(_idx=0,
                              _sym="SA_{TAS_{0}}",
                              _fwk="SOFTWARE",
                              name="Sensitivity Analysis TAS No 1",
                              description="Sensitivity Analysis for TAS 1",
                              _variables=tas_vars,
                              _coefficients=tas_model._coefficients,)

In [9]:
for k, v in tas_sena.coefficients.items():
    pi = v
    print(f"Coefficient: {k} = {pi._pi_expr}")

Coefficient: \Pi_{0} = \frac{n_{1}}{K_{1}}
Coefficient: \Pi_{1} = \lambda_{1}*W_{1}
Coefficient: \Pi_{2} = \frac{\miu_{1}}{\lambda_{1}}
Coefficient: \Pi_{3} = \frac{L_{1}}{K_{1}}


In [12]:
print("=== Sym Analysis: ===")
tas_sena.analyze_symbolic(val_type="mean")

for k, v in tas_sena.results.items():
    print(f"{k}: {v}")
# print(tas_sena.results, "\n")

print("\n=== Num Analysis: ===")
tas_sena.analyze_numeric(n_samples=70000)

for k, v in tas_sena.results.items():
    print(f"{k}: {v}")
    for a, b in v.items():
        print(f"\t{a}: {b}")

print("Sensitivity Analysis Results:")
# print(tas_sena.analyses, "\n")

for key, val in tas_sena.results.items():
    txt = f"{key}: {val}"
    print(txt)

=== Sym Analysis: ===
SEN_{\Pi_{0}}: {'K_{1}': -0.003, 'n_{1}': 0.01}
SEN_{\Pi_{1}}: {'W_{1}': 344.83, '\\lambda_{1}': 30.0}
SEN_{\Pi_{2}}: {'\\lambda_{1}': -0.008409882261236259, '\\miu_{1}': 0.0028999797001420993}
SEN_{\Pi_{3}}: {'K_{1}': -0.003, 'L_{1}': 0.01}

=== Num Analysis: ===
SEN_{\Pi_{0}}: {'S1': [0.0027039587751457117, 7.941144127168429e-05], 'ST': [0.8753211260830042, 0.6507768357371457], 'S1_conf': [0.00013624010984825295, 0.00015587400270061482], 'ST_conf': [0.0020197866922503317, 0.0013976553755172345], 'names': ['K_{1}', 'n_{1}']}
	S1: [0.0027039587751457117, 7.941144127168429e-05]
	ST: [0.8753211260830042, 0.6507768357371457]
	S1_conf: [0.00013624010984825295, 0.00015587400270061482]
	ST_conf: [0.0020197866922503317, 0.0013976553755172345]
	names: ['K_{1}', 'n_{1}']
SEN_{\Pi_{1}}: {'S1': [0.4277475084955373, 0.42745567971132664], 'ST': [0.5712670020092611, 0.5715594966537534], 'S1_conf': [0.0002474287111384245, 0.0002471331351756364], 'ST_conf': [0.00399067369403694, 

#### **Monte Carlo Simulation**

In [13]:
print("\n=== monte carlo handler ===")
# Create a handler
tas_mch = MonteCarloHandler(_sym="MCH_{0}",
                            _fwk="SOFTWARE",
                            name="TAS Monte Carlo Handler",
                            description="TAS Monte Carlo Handler for simulations",
                            _variables=tas_model.relevant_lt,
                            _coefficients=tas_model.coefficients)
print(tas_mch, "\n")


=== monte carlo handler ===
MonteCarloHandler(sym='MCH_{0}', alias='MCH_0', fwk='SOFTWARE', idx=-1, name='TAS Monte Carlo Handler', description='Tas monte carlo handler for simulations', cat='NUM', variables={'\\lambda_{1}': Variable(_sym='\\lambda_{1}', _alias='lambda_tas_1', _fwk='SOFTWARE', _idx=0, name='TAS component 1 arrival 1', description='Tas arrival rate 1', _cat='IN', _dims='E*T^-1', _units='req/s', _std_dims='T^(-1)*E^(1)', _sym_exp='T**(-1)* E**(1)', _dim_col=[-1, 0, 1, 0, 0], _min=0.0, _max=344.83, _mean=344.83, _dev=None, _std_units='req/s', _std_min=0.0, _std_max=344.83, _std_mean=344.83, _std_dev=None, _step=0.001, _std_range=array([], dtype=float64), _dist_type='exponential', _dist_params={'scale': 0.0028999797001420993}, _dist_func=<function <lambda> at 0x000001F9DB39E830>, relevant=True), 'K_{1}': Variable(_sym='K_{1}', _alias='k_1', _fwk='SOFTWARE', _idx=1, name='TAS component 1 max queue size', description='Tas max queue size 1', _cat='IN', _dims='E', _units='req

In [ ]:
tas_mch._create_distributions()
print(tas_mch._distributions)
tas_mch._create_simulations()
print(tas_mch._simulations.keys())
for k, v in tas_mch._simulations.items():
    print(f"Simulation dist = {k}:\n\t{v._distributions}")
tas_mch.simulate(n_samples=70000)

{'\\lambda_{1}': {'dtype': 'exponential', 'params': {'scale': 0.0028999797001420993}, 'func': <function <lambda> at 0x000001F9DB39E830>}, 'K_{1}': {'dtype': 'uniform', 'params': {'constant': 100}, 'func': <function <lambda> at 0x000001F9DB39FAC0>}, 'n_{1}': {'dtype': 'uniform', 'params': {'low': 0, 'high': 60}, 'func': <function <lambda> at 0x000001F9DB39FB50>}, 'W_{1}': {'dtype': 'uniform', 'params': {'low': 0, 'high': 60}, 'func': <function <lambda> at 0x000001F9DB39FBE0>}, '\\miu_{1}': {'dtype': 'exponential', 'params': {'scale': 0.001}, 'func': <function <lambda> at 0x000001F9DB39FA30>}, 'L_{1}': {'dtype': 'uniform', 'params': {'low': 0, 'high': 60}, 'func': <function <lambda> at 0x000001F9DB39FC70>}}
dict_keys(['\\Pi_{0}', '\\Pi_{1}', '\\Pi_{2}', '\\Pi_{3}'])
Simulation dist = \Pi_{0}:
	{'K_{1}': {'dtype': 'uniform', 'params': {'constant': 100}, 'func': <function <lambda> at 0x000001F9DB39FAC0>}, 'n_{1}': {'dtype': 'uniform', 'params': {'low': 0, 'high': 60}, 'func': <function <la

In [ ]:
print("=== Monte Carlo Simulation Report ===")
for pi, results in tas_mch._results.items():
    for k, v in results.items():
        if k == "statistics":
            coef = tas_mch._coefficients[pi].pi_expr
            print(f"\nCoefficient: {pi} = {coef}")
            # print(f"Result for {k}: {v}")
            stats = v
            for a, b in stats.items():
                print(f"\t{a}: {b}")
        # else:
        #     print(f"Other result for {k}: {v.shape}")


Coefficient: \Pi_{0} = \frac{n_{1}}{K_{1}}
	mean: 17.286683817680917
	median: 3.3534877295289043
	std_dev: 293.28988932342196
	variance: 86018.95917934511
	min: 1.6666736987872606
	max: 34964.80071391566
	count: 70000

Coefficient: \Pi_{1} = \lambda_{1}*W_{1}
	mean: 0.08698115393652564
	median: 0.046201027903008986
	std_dev: 0.11323148407003614
	variance: 0.012821368984702848
	min: 4.0021059756658364e-08
	max: 1.4752363019172134
	count: 70000

Coefficient: \Pi_{2} = \frac{\miu_{1}}{\lambda_{1}}
	mean: 28.594635182573015
	median: 2.9184385406800626
	std_dev: 1007.9755704049244
	variance: 1016014.7505331327
	min: 7.048222393726727e-05
	max: 232076.5106156933
	count: 70000

Coefficient: \Pi_{3} = \frac{L_{1}}{K_{1}}
	mean: 20.792941850447672
	median: 3.3244814781413785
	std_dev: 615.8177564732781
	variance: 379231.5091877817
	min: 1.6667261614614328
	max: 105135.10506085868
	count: 70000


### **Arternative Design No.1 (Horizontal Scaling)**

#### **Dimensional Variables**

#### **Dimensionless Coefficients**

#### **Sensitivity Analysis**

#### **Monte Carlo Simulation**

### **Arternative Design No.2 (Vertical Scaling)**

#### **Dimensional Variables**

#### **Dimensionless Coefficients**

#### **Sensitivity Analysis**

#### **Monte Carlo Simulation**

## **Conclusion**

## **Future Work**